In [26]:
import pandas as pd
import re
from utils import mahalanobis_algorithm, valid_params, prepare_params, load_properties

In [46]:
df = pd.read_json("assets/olx.json", orient="records", convert_dates=False)
df = df.drop(["titulo", "link", "descricao", "created_at", "codigo"], axis=1)
df = pd.concat([df.drop(["_id", "data_publicacao", "caracteristicas"], axis=1), df['caracteristicas'].apply(pd.Series)], axis=1)
df = df.drop(["tipo", "cep", "municipio", "logradouro", "detalhes_do_imovel", "detalhes_do_condominio", "condominio", "iptu"], axis=1)
df = df.dropna()
df = df.rename({'preco': 'value', 'quartos': 'rooms', 'area_util': 'area'}, axis=1)

df['value'] = df['value'].apply(lambda x: re.sub('[^0-9]','', x))
df['value'] = df['value'].astype(float)
df['rooms'] = df['rooms'].apply(lambda x: re.sub('[^0-9]','', x))
df['rooms'] = df['rooms'].astype(float)
df['area'] = df['area'].apply(lambda x: re.sub('[^0-9]','', x))
df['area'] = df['area'].astype(float)
df

,value,categoria,area,rooms,banheiros,vagas_na_garagem,bairro
3,200001.0,Apartamentos,61.0,2.0,2,1,Jabotiana
6,430000.0,Apartamentos,0.0,2.0,2,1,Atalaia
11,480000.0,Apartamentos,140.0,4.0,4,2,Salgado Filho
12,95000.0,Apartamentos,45.0,2.0,1,1,Lamarão
13,940019.0,Apartamentos,140.0,3.0,4,2,Grageru
...,...,...,...,...,...,...,...
5073,156000.0,Apartamentos,0.0,2.0,0,0,São José
5074,220000.0,Apartamentos,78.0,3.0,1,1,Farolândia
5075,280050.0,Apartamentos,123.0,4.0,3,1,Luzia
5077,0.0,Apartamentos,64.0,3.0,2,1,Inácio Barbosa


In [47]:
params = {'value': 10000.0, 'rooms': 3.0, 'area': 0.5}

weights = {'value': [10.0], 'rooms': [3.0], 'area': [0.5]}

sorted_values = mahalanobis_algorithm(df, params, weights)

In [48]:
sorted_values.iloc[2].name

df.iloc[sorted_values.iloc[2].name]

value                          0
categoria           Apartamentos
area                           0
rooms                          3
banheiros                      0
vagas_na_garagem               0
bairro                   Grageru
Name: 576, dtype: object

In [49]:
df['bairro'].unique()

array(['Jabotiana', 'Atalaia', 'Salgado Filho', 'Lamarão', 'Grageru',
       'Luzia', 'Jardins', 'Farolândia', 'Coroa do Meio', 'Ponto Novo',
       'Suíssa', 'Centro', 'São Conrado', 'Inácio Barbosa', 'São José',
       'Zona de Expansão (Mosqueiro)', 'Treze de Julho',
       'Dezoito do Forte', 'Aeroporto', 'Zona de Expansão (Aruana)',
       'Siqueira Campos', 'Pereira Lobo', 'Jardim Centenário',
       'Santa Maria', 'Dom Luciano', 'Olaria', 'Santos Dumont', 'América',
       'Santo Antônio', 'Getúlio Vargas', 'Cirurgia', 'Industrial',
       "Porto D'Antas", '18 do Forte', 'José Conrado de Araújo',
       'Cidade Nova', 'Zona de Expansão (Robalo)', '13 De Julho',
       'Palestina', 'Soledade', 'Japãozinho'], dtype=object)